# Importing the libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models , initializers
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.optimizers import legacy
import pandas as pd
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import shutil

# load data

In [ ]:
def load_images_from_dir(directory, target_size=(320, 320)):
    image_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(directory, filename)
            image = cv2.imread(image_path)
            if image is None:
                print(f"Warning: Unable to load image '{filename}'")
                continue
            
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            resized_image = cv2.resize(image_rgb, target_size)
            image_list.append(resized_image)
    return image_list



In [ ]:
directory_path = "data/monet_jpg/"
monet_image_list = load_images_from_dir(directory_path)
print("Number of images loaded:", len(monet_image_list))

In [ ]:
directory_path = "data/photo_jpg/"
image_list = load_images_from_dir(directory_path)
print("Number of images loaded:", len(image_list))

# EDA images

In [ ]:
def plot_random_images(image_list):
    if len(image_list) < 5:
        print("Error: Insufficient number of images.")
        return
    random_indices = random.sample(range(len(image_list)), 5)
    
    fig, axes = plt.subplots(1, 5, figsize=(15, 5))
    for i, idx in enumerate(random_indices):
        axes[i].imshow(image_list[idx])  
        axes[i].axis('off')
        axes[i].set_title(f"Image {idx}")
    plt.show()


In [ ]:
plot_random_images(image_list)

In [ ]:
plot_random_images(monet_image_list)

# cleaning the data

In [ ]:
def has_white_corner(image):
    # Convert the image to grayscale for corner detection
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Define the size of the corner area to check
    corner_size = 5
    
    # Extract corner regions of the image
    corners = [
        gray_image[:corner_size, :corner_size],                # Top-left corner
        gray_image[:corner_size, -corner_size:],              # Top-right corner
        gray_image[-corner_size:, :corner_size],              # Bottom-left corner
        gray_image[-corner_size:, -corner_size:]              # Bottom-right corner
    ]
    
    # Check if all corners are white (brightness value over 220)
    return all(np.all(corner > 220) for corner in corners)

def crop_and_resize(image, target_size=(320, 320)):
    # Get dimensions of the original image
    height, width = image.shape[:2]
    
    # Calculate the center point of the image
    center_x = width // 2
    center_y = height // 2
    
    # Define the cropping area centered at the image's center
    crop_left = max(0, center_x - 215 // 2)
    crop_top = max(0, center_y - 215 // 2)
    crop_right = min(width, center_x + 215 // 2)
    crop_bottom = min(height, center_y + 215 // 2)
    
    # Crop and resize the image to the target size
    cropped_image = image[crop_top:crop_bottom, crop_left:crop_right]
    resized_image = cv2.resize(cropped_image, target_size)
    
    return resized_image

def show_images_with_white_corner(images):
    # Iterate through each image in the list
    for idx, image in enumerate(images):
        # Check if the image has white corners
        if has_white_corner(image):
            # Crop and resize the image if it has white corners
            cropped_resized_image = crop_and_resize(image)
            # Display the original image
            plt.imshow(image)
            plt.title(f"Image {idx}")
            plt.axis('off')
            plt.show()

            # Display the cropped and resized image
            plt.imshow(cropped_resized_image)
            plt.title(f"Cropped & Resized Image {idx}")
            plt.axis('off')
            plt.show()

# Assuming `monet_image_list` is a predefined list of images
show_images_with_white_corner(monet_image_list)


### we notice that there are a few round images with white corners. We can crop and resize these images to remove the white corners.

In [ ]:
def change_images_with_white_corner(images):
    new_images = []
    for idx, image in enumerate(images):
        if has_white_corner(image):
            new_images.append(crop_and_resize(image))
        else:
            new_images.append(image)

    return new_images

monet_image_list = change_images_with_white_corner(monet_image_list)

# Augmentation of the data

In [ ]:
import numpy as np
import cv2

def split_and_resize_quarters(original_image):
    """
    Splits an image into quarters and resizes each quarter back to 320x320 pixels.
    
    Parameters:
        original_image: A PIL Image object of size 320x320.
    
    Returns:
        A tuple containing the original image and a list of the resized quarter images.
        If the original image is not 320x320, it returns None.
    """
    
    # Convert PIL Image to NumPy array for processing
    original_image_array = np.array(original_image)
    
    # Ensure the image size is exactly 320x320 pixels
    if original_image_array.shape[:2] != (320, 320):
        print("Error: Image size must be 320x320.")
        return None
    
    # Initialize an empty list to store the quarters of the image
    quarters = []
    
    # Loop to divide the image into 4 quarters
    for i in range(2):  # Rows
        for j in range(2):  # Columns
            # Calculate the coordinates for each quarter
            left = j * 160  # x-coordinate of the left edge
            upper = i * 160  # y-coordinate of the top edge
            right = left + 160  # x-coordinate of the right edge
            lower = upper + 160  # y-coordinate of the bottom edge
            
            # Extract the quarter using calculated coordinates
            quarter = original_image_array[upper:lower, left:right, :]
            
            # Add the extracted quarter to the list
            quarters.append(quarter)
    
    # Resize each quarter back to 320x320 pixels
    resized_quarters = [cv2.resize(quarter, (320, 320)) for quarter in quarters]
    
    # Return the original image and the list of resized quarter images
    return original_image, resized_quarters


In [ ]:
# Assuming split_and_resize_quarters function is defined and monet_image_list is a list of images
original_image, quarters = split_and_resize_quarters(monet_image_list[2])

# Check if the quarters list is not empty
if quarters:
    # Set up a figure for displaying the original image
    plt.figure(figsize=(8, 8))
    plt.imshow(original_image)  # Display the original image
    plt.axis('off')  # Remove axes for clarity
    plt.title("Original Image")  # Set the title of the figure
    plt.show()  # Display the figure

    # Set up a 2x2 subplot structure for displaying the quarter images
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    
    # Loop through each quarter image for plotting
    for i, quarter in enumerate(quarters):
        # Determine the position of the subplot using integer division and modulus
        row = i // 2  # Row index for the subplot
        col = i % 2   # Column index for the subplot
        
        # Display the quarter image in the appropriate subplot
        axes[row, col].imshow(quarter)
        axes[row, col].axis('off')  # Remove axes for clarity
        axes[row, col].set_title(f"Quarter {i+1}")  # Set the title for each subplot

    plt.show()  # Display all the subplots


In [ ]:
def get_images_with_high_color_variance(images, threshold):
    """
    Filters a list of images, returning those with a color variance higher than a specified threshold.
    
    Parameters:
        images (list): A list of image objects, which can be converted into NumPy arrays.
        threshold (float): The minimum color variance required for an image to be included in the return list.

    Returns:
        list: A list of images that have a color variance greater than the threshold.
    """
    
    # Initialize an empty list to store images with high color variance
    high_color_var_images = []
    
    # Iterate through each image in the provided list
    for image in images:
        # Convert the image to a NumPy array for analysis
        image_array = np.array(image)
        
        # Calculate the variance of the colors in the image
        color_variance = np.var(image_array)
        
        # If the color variance is greater than the threshold, add the image to the result list
        if color_variance > threshold:
            high_color_var_images.append(image)
    
    # Return the list of images with high color variance
    return high_color_var_images


In [ ]:
most_colorful_image = get_images_with_high_color_variance(quarters , 2500)
if most_colorful_image:
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    for i, quarter in enumerate(most_colorful_image):
        axes[i//2, i%2].imshow(quarter)
        axes[i//2, i%2].axis('off')
    plt.show()

# we notice that we dont have engoh data in order to train the model, so we need to augment the data, here we just take the same iamge and flip it to get more data

In [ ]:
def mirror_flip(image):
    flipped_image = cv2.flip(image, 1)
    return flipped_image


def aug_pipeline(images, threshold):
    augmented_images = []
    
    for image in images:
        _ , quarters = split_and_resize_quarters(image)
        
        important_quarters = get_images_with_high_color_variance(quarters, threshold)

        mirrored_image = mirror_flip(image)
                
        augmented_images.append(image)
        augmented_images.append(mirrored_image)
        if len(important_quarters) != 0:
            augmented_images.extend(important_quarters)
    
    return augmented_images

In [ ]:
print(len(monet_image_list))
augmented_images = aug_pipeline(monet_image_list, 2500)
print(len(augmented_images))
plot_random_images(augmented_images)

# save the data in batch for model

In [ ]:
# for debug
# augmented_images = augmented_images[:50]
# image_list = image_list[:100]

In [ ]:
def decode_image(image):
    """
    Decodes an image by normalizing its pixel values to the range [-1, 1] and reshaping it.

    This is commonly done as a preprocessing step before feeding the image to a neural network
    to standardize the range of input values and ensure consistent input dimensions.

    Parameters:
        image: A tensor representing the image.

    Returns:
        A tensor of the image with pixel values normalized and reshaped to [320, 320, 3].
    """
    # Normalize the image pixel values to the range [-1, 1]
    # This helps in stabilizing the training process in neural networks.
    image = (tf.cast(image, tf.float32) / 127.5) - 1

    # Reshape the image to the expected input size of the neural network, which is 320x320 with 3 color channels.
    # This ensures the image has a consistent size for processing in neural networks.
    image = tf.reshape(image, [320, 320, 3])

    return image

def encode_image(image):
    """
    Encodes an image by converting its pixel values from the range [-1, 1] back to [0, 255].

    This is typically done after processing an image through a neural network that outputs normalized pixel values,
    to convert it back to the original pixel value range for displaying or saving as an image file.

    Parameters:
        image: A tensor representing the image with pixel values in the range [-1, 1].

    Returns:
        A tensor of the image with pixel values in the range [0, 255] as unsigned 8-bit integers.
    """
    # Reverse the normalization process to convert the pixel values back to the range [0, 255].
    # This is necessary for converting the output of neural networks back to a standard image format.
    image = (image + 1) * 127.5

    # Clip the pixel values to ensure they remain within the valid range [0, 255].
    # This is important to avoid any values outside the range due to numeric computations.
    image = tf.clip_by_value(image, 0, 255)

    # Convert the pixel values back to unsigned 8-bit integers, which is a standard format for images.
    # This is necessary for saving the image in a standard image file format like JPEG or PNG.
    image = tf.cast(image, tf.uint8)

    return image


In [ ]:
image_list = random.sample(image_list, len(augmented_images))

In [ ]:
# Set the batch size for the dataset, used in training the model in batches
BATCH_SIZE = 10 

# Calculate the length of the augmented images and the real image list
monet_len, real_len = len(augmented_images), len(image_list)

# Decode the images using the previously defined decode_image function
# This normalizes the pixel values and reshapes the images
decoded_monet_images = [decode_image(image) for image in augmented_images]
decoded_real_images = [decode_image(image) for image in image_list]

# Print the lengths of the decoded images lists to verify the decoding process
print(len(decoded_monet_images), monet_len)
print(len(decoded_real_images), real_len)

# Split the decoded images into training, validation, and test sets
# Here, 80% of the images are used for training, 10% for validation, and 10% for testing
train_monet = decoded_monet_images[:int(0.8 * monet_len)]
val_monet = decoded_monet_images[int(0.8 * monet_len):int(0.9 * monet_len)]
test_monet = decoded_monet_images[int(0.9 * monet_len):]

train_real = decoded_real_images[:int(0.8 * real_len)]
val_real = decoded_real_images[int(0.8 * real_len):int(0.9 * real_len)]
test_real = decoded_real_images[int(0.9 * real_len):]

# Create TensorFlow datasets from the image lists to efficiently manage memory and training speed
train_real = tf.data.Dataset.from_tensor_slices(train_real)
val_real = tf.data.Dataset.from_tensor_slices(val_real)
test_real = tf.data.Dataset.from_tensor_slices(test_real)

train_monet = tf.data.Dataset.from_tensor_slices(train_monet)
val_monet = tf.data.Dataset.from_tensor_slices(val_monet)
test_monet = tf.data.Dataset.from_tensor_slices(test_monet)

# Pair the real and Monet datasets for training, validation, and testing
train_dataset = tf.data.Dataset.zip((train_real, train_monet))
val_dataset = tf.data.Dataset.zip((val_real, val_monet))
test_dataset = tf.data.Dataset.zip((test_real, test_monet))

# Shuffle and batch the datasets for training
# Shuffling helps in reducing variance and making sure that the model remains general and overfits less
train_dataset = train_dataset.shuffle(buffer_size=len(train_real)).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# Individual datasets are also shuffled and batched for potential separate use or evaluation
train_monet = train_monet.shuffle(buffer_size=len(train_monet)).batch(BATCH_SIZE)
train_real = train_real.shuffle(buffer_size=len(train_real)).batch(BATCH_SIZE)
val_monet = val_monet.shuffle(buffer_size=len(val_monet)).batch(BATCH_SIZE)
val_real = val_real.shuffle(buffer_size=len(val_real)).batch(BATCH_SIZE)


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

def plot_images_from_batch(dataset, title, img_to_plot=10):
    """
    Plots a specified number of images from both Monet and real image batches in a dataset.

    Parameters:
        dataset: A tf.data.Dataset object containing batches of Monet and real images.
        title: A string to set as the title of the plotted figure.
        img_to_plot: An integer specifying how many images to plot from each batch.
    """
    # Take one batch from the dataset
    for real_batch, monet_batch in dataset.take(1):  
        # Initialize the figure with a specified size
        plt.figure(figsize=(12, 8))

        # Loop through the number of images specified to plot from the batch
        for i in range(img_to_plot):
            # Plot Monet images in the top row
            plt.subplot(2, img_to_plot, i + 1)
            monet_image = encode_image(monet_batch[i])  # Encode the image back to displayable format
            plt.imshow(monet_image)  # Display the Monet image
            plt.title("Monet")  # Set title for the Monet image subplot
            plt.axis("off")  # Hide axes for cleaner visualization

            # Plot real images in the bottom row
            plt.subplot(2, img_to_plot, i + 1 + img_to_plot)
            real_image = encode_image(real_batch[i])  # Encode the image back to displayable format
            plt.imshow(real_image)  # Display the real image
            plt.title("Real")  # Set title for the real image subplot
            plt.axis("off")  # Hide axes for cleaner visualization

        # Set the main title for the figure and display it
        plt.suptitle(title)
        plt.show()

# Example usage, plotting 5 images from each of the Monet and real image batches in the training dataset
plot_images_from_batch(train_dataset, "Monet and Real Images", img_to_plot=5)


# create the objects that neccessary for the training

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
    """
    Creates a downsampling layer for a convolutional neural network.

    Parameters:
        filters: Number of filters in the convolutional layer.
        size: Kernel size for the convolutional layer.
        apply_batchnorm: Boolean flag to determine whether to apply batch normalization.

    Returns:
        A TensorFlow Sequential model containing the downsampling layers.
    """
    # Initialize the weights with a normal distribution for the convolutional layer
    initializer = tf.random_normal_initializer(0., 0.02)

    # Create a sequential model to hold the downsampling layers
    result = tf.keras.Sequential()

    # Add a convolutional layer with specified filters and kernel size
    # Stride of 2 for downsampling, 'same' padding to keep dimensions consistent
    result.add(
        tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                               kernel_initializer=initializer, use_bias=False))

    # Optionally add batch normalization to improve training stability and performance
    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    # Add Leaky ReLU activation to introduce non-linearity and allow gradients to flow even for negative values
    result.add(tf.keras.layers.LeakyReLU())

    return result

# Getting a sample from the training dataset
sample, _ = next(iter(train_dataset))
# Casting the sample to float32, typically needed for model processing in TensorFlow
sample_float32 = tf.cast(sample, tf.float32)

# Create a downsampling model with 3 filters and kernel size 4
down_model = downsample(3, 4)
# Apply the downsampling model to the sample
down_result = down_model(sample_float32)

# Print the shape of the output from the downsampling model to verify the result
print(down_result.shape)


In [ ]:
def upsample(filters, size, apply_dropout=False):
    """
    Creates an upsampling layer for a convolutional neural network.

    Parameters:
        filters: Number of filters in the transposed convolutional layer.
        size: Kernel size for the transposed convolutional layer.
        apply_dropout: Boolean flag to determine whether to apply dropout.

    Returns:
        A TensorFlow Sequential model containing the upsampling layers.
    """
    # Initialize the weights with a normal distribution for the transposed convolutional layer
    initializer = tf.random_normal_initializer(0., 0.02)

    # Create a sequential model to hold the upsampling layers
    result = tf.keras.Sequential()

    # Add a transposed convolutional layer (also known as deconvolution)
    # which increases the spatial dimensions of the input
    result.add(
        tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                        padding='same',
                                        kernel_initializer=initializer,
                                        use_bias=False))

    # Always add batch normalization to standardize the activations from the previous layer,
    # which helps to speed up training and reduce the number of training epochs required
    result.add(tf.keras.layers.BatchNormalization())

    # Optionally add dropout to prevent overfitting by randomly setting a fraction of input units to 0
    # at each update during training time, which helps to prevent overfitting
    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    # Add ReLU activation function to introduce non-linearity to the model and allow it to learn more complex patterns
    result.add(tf.keras.layers.ReLU())

    return result

# Example of using the upsample function to create a model and apply it to an input
up_model = upsample(3, 4)  # Create an upsampling model with 3 filters and kernel size 4
up_result = up_model(down_result)  # Apply the upsampling model to the result of a previous downsampling operation
print(up_result.shape)  # Print the shape of the output to verify the upsampling effect


In [ ]:
# Define Generator model
def Generator():
# Input layer of the generator with shape 320x320x3
  inputs = tf.keras.layers.Input(shape=[320, 320, 3])
  
  # Define the downsampling part of the generator
  down_stack = [
    downsample(80, 4, apply_batchnorm=False),  # First layer without batchnorm, reduces size to 160x160
    downsample(160, 4),  # Reduces size to 80x80
    downsample(320, 4),  # Reduces size to 80x80
    downsample(640, 4), # Reduces size to 80x80
    downsample(640, 4),  # Reduces size to 10x10
    downsample(640, 4),  # Reduces size to 5x5
    # downsample(640, 4), 
    # downsample(640, 4),  
  ]

  up_stack = [
    # upsample(640, 4, apply_dropout=True),  
    # upsample(640, 4, apply_dropout=True), 
    upsample(640, 4, apply_dropout=True),  # Extends size to 10x10, uses dropout to prevent overfitting
    upsample(640, 4 ,apply_dropout=True),  # Extends size to 20x20
    upsample(320, 4),  # Extends size to 40x40
    upsample(160, 4),  # Extends size to 80x80
    upsample(80, 4),   # Extends size to 160x160
  ]
    
  # Output layer of the generator
  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(3, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')  # Extends size to 320x320, output 3 channels


  # Process the input through the down_stack
  x = inputs
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

   # Prepare skip connections for the upsampling layers
  skips = reversed(skips[:-1]) # Reverse and exclude the last downsampled output
   # Process through the up_stack using skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])# Concatenate skip connection with the upsampled output
  # Final layer to generate the output image
  x = last(x)
    
  # Create the model  
  return tf.keras.Model(inputs=inputs, outputs=x)

def test_build_generator():

    # Instantiate the Generator
    generator = Generator()
    # Define the input shape and generate a batch of random noise
    input_shape = (320, 320, 3) 
    batch_size = 4
    random_input_batch = tf.random.uniform((batch_size,) + input_shape, minval=0, maxval=1)
    
    # Generate images from the random noise
    generated_images = generator(random_input_batch)
    print(generator.summary())
    # Check the output shape of the generated images
    output_shape = generated_images.shape
    expected_output_shape = (batch_size, 320, 320, 3)  
    # Assert the output shape is as expected
    assert output_shape == expected_output_shape, "Output shape mismatch"
    print("Test passed!")

test_build_generator()

In [ ]:
def Discriminator(input_nc=3, ndf=80, n_layers=3):
    """
    Defines the Discriminator model for a GAN, which classifies images as real or fake.

    Parameters:
        input_nc: Number of channels in the input images, typically 3 for RGB images.
        ndf: Number of filters in the first convolutional layer, controls model capacity.
        n_layers: Number of layers in the discriminator.

    Returns:
        A TensorFlow model representing the discriminator.
    """
    # Input layer of the discriminator with shape 320x320x(input_nc)
    inputs = tf.keras.layers.Input(shape=(320, 320, input_nc))
    x = inputs

    # Build the convolutional layers of the discriminator
    for n in range(n_layers + 1):
        # Increase the number of filters as we go deeper, capped at 5 times the initial number
        nf_mult = min(2**n, 5)
        # Add a convolutional layer with increasing number of filters
        x = tf.keras.layers.Conv2D(ndf * nf_mult, kernel_size=4, strides=2, padding='same')(x)
        # Use LeakyReLU to allow small gradients for negative values, improving training stability
        x = tf.keras.layers.LeakyReLU(0.2)(x)
        # Apply batch normalization except for the last layer to help stabilize training
        if n < n_layers:
            x = tf.keras.layers.BatchNormalization()(x)

    # Output layer: a convolutional layer that produces a single output channel
    outputs = tf.keras.layers.Conv2D(1, kernel_size=4, strides=1, padding='same')(x)

    # Construct and return the discriminator model
    return tf.keras.Model(inputs=inputs, outputs=outputs)

def test_build_discriminator():
    # Define the input shape for the test
    input_shape = (320, 320, 3) 

    # Create a discriminator model
    discriminator = Discriminator(3)
    
    # Define a batch size for testing
    batch_size = 4
    # Generate a batch of random input data
    random_input_batch = tf.random.uniform((batch_size,) + input_shape, minval=0, maxval=1)
    
    # Get the discriminator output for the random input
    discriminator_output = discriminator(random_input_batch)
    
    # Check and print the output shape
    output_shape = discriminator_output.shape
    print(output_shape)
    # Print the discriminator model summary
    print(discriminator.summary())
    # Define the expected output shape
    expected_output_shape = (batch_size, 20, 20, 1)  
    
    # Assert the output shape matches the expected shape
    assert output_shape == expected_output_shape, "Output shape mismatch"
    print("Test passed!")

# Run the test function to build and validate the discriminator
test_build_discriminator()


# step 1

In [ ]:
import tensorflow as tf
import pandas as pd
from tqdm import tqdm  

# Initialize the optimizer; legacy.Adam() suggests using a version of Adam optimizer with some custom settings or from a previous version
optimizer = tf.keras.optimizers.Adam()

# Define the loss function; MeanSquaredError is commonly used for regression tasks and measures the average of the squares of the errors
loss_function = tf.keras.losses.MeanSquaredError()

def train_model(model, train_dataset, val_dataset, optimizer, loss_function, num_epochs):
    """
    Trains a machine learning model using the specified optimizer and loss function.

    Parameters:
        model: The neural network model to train.
        train_dataset: The dataset for training the model.
        val_dataset: The dataset for validating the model performance.
        optimizer: The optimization algorithm to use for training.
        loss_function: The loss function to use for training.
        num_epochs: The number of epochs to train the model.

    Returns:
        A DataFrame containing the training and validation losses for each epoch.
    """
    # Compile the model with the given optimizer and loss function
    model.compile(optimizer=optimizer, loss=loss_function)
    
    # Lists to store the loss values for each epoch
    train_losses = []
    val_losses = []
    
    # Train the model for a specified number of epochs
    for epoch in range(num_epochs):
        train_epoch_loss = 0
        val_epoch_loss = 0
        
        # Iterate over each batch in the training dataset
        for batch in tqdm(train_dataset, desc=f'Epoch {epoch}'):
            # Record operations for automatic differentiation
            with tf.GradientTape() as tape:
                generated_images = model(batch, training=True)
                loss = loss_function(batch, generated_images)
            # Compute gradients of the loss with respect to the model parameters
            gradients = tape.gradient(loss, model.trainable_variables)
            # Apply gradients to update the model parameters
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            train_epoch_loss += loss
            
        # Iterate over each batch in the validation dataset
        for val_batch in tqdm(val_dataset, desc=f'Epoch {epoch}'):
            val_generated_images = model(val_batch, training=False)
            val_loss = loss_function(val_batch, val_generated_images)
            val_epoch_loss += val_loss
        
        # Calculate the average loss for the epoch
        train_epoch_loss /= len(train_dataset)
        val_epoch_loss /= len(val_dataset)
        
        # Append the average loss for each epoch to the respective lists
        train_losses.append(train_epoch_loss.numpy())
        val_losses.append(val_epoch_loss.numpy())
        
        # Print the loss values for the epoch
        print(f'Epoch {epoch + 1}, Training Loss: {train_epoch_loss.numpy()}, Validation Loss: {val_epoch_loss.numpy()}')
    
    # Create a DataFrame to store and display the loss values for each epoch
    losses_df = pd.DataFrame({'Epoch': range(1, num_epochs + 1), 'Training Loss': train_losses, 'Validation Loss': val_losses})
    
    return losses_df

# Set the number of epochs for training
num_epochs = 20

# Train the model for the 'Monet' generator
print("Training monet_gen:")
monet_gen_trained = Generator()  # Instantiate the generator model
monet_losses = train_model(monet_gen_trained, train_monet, val_monet, optimizer, loss_function, num_epochs)  # Train the model

# Train the model for the 'Real' generator
print("Training real_gen:")
real_gen_trained = Generator()  # Instantiate another generator model
real_losses = train_model(real_gen_trained, train_real, val_real, optimizer, loss_function, num_epochs)  # Train the model


In [ ]:
# Save the trained weights of the real image generator
# This allows the model's learned parameters to be reused without retraining from scratch.
# 'generator_m2r.h5' is the filename, where 'm2r' indicates Monet to Real conversion.
real_gen_trained.save_weights(os.path.join("pre_train", f'generator_m2r.h5'))

# Save the trained weights of the Monet-style image generator
# Similar to above, 'generator_r2m.h5' is the filename, where 'r2m' indicates Real to Monet conversion.
monet_gen_trained.save_weights(os.path.join("pre_train", f'generator_r2m.h5'))


In [ ]:
# Save the losses DataFrame to a CSV file for both models.
# This is useful for later analysis of the training process, such as plotting loss curves.
# 'monet_losses.csv' stores the loss data for the Monet generator.
monet_losses.to_csv('pre_train/monet_losses.csv', index=False)
# 'real_losses.csv' stores the loss data for the real image generator.
real_losses.to_csv('pre_train/real_losses.csv', index=False)

In [ ]:
# Load the training and validation loss data for the Monet-style generator from a CSV file.
# This data is used for analyzing the model's performance over the training period.
monet_losses = pd.read_csv('pre_train/monet_losses.csv')

# Load the training and validation loss data for the real-image generator from a CSV file.
# Similar to the Monet generator, this data helps in evaluating the training effectiveness.
real_losses = pd.read_csv('pre_train/real_losses.csv')

# Instantiate new generator models.
# These generators are neural network models that were previously trained to convert images.
monet_gen_trained = Generator()  # For generating Monet-styled images
real_gen_trained = Generator()   # For generating real-styled images

# Load the saved weights into the instantiated models.
# This step effectively restores the models to their trained state,
# allowing them to generate images without needing to be retrained.

# Load weights for the real image generator (Monet to Real conversion).
# The weights file 'generator_m2r.h5' contains the learned parameters of the model.
real_gen_trained.load_weights(os.path.join("pre_train", f'generator_m2r.h5'))

# Load weights for the Monet-style image generator (Real to Monet conversion).
# The weights file 'generator_r2m.h5' stores the trained parameters of the model.
monet_gen_trained.load_weights(os.path.join("pre_train", f'generator_r2m.h5'))


In [ ]:
# Start a new matplotlib figure with specified size
plt.figure(figsize=(10, 6))

# Plot the training loss per epoch.
# 'Epoch' column from the 'monet_losses' DataFrame is used as the x-axis,
# and 'Training Loss' column as the y-axis.
# This line graph represents how the training loss changes over epochs.
plt.plot(monet_losses['Epoch'], monet_losses['Training Loss'], label='Training Loss')

# Similarly, plot the validation loss per epoch.
# 'Validation Loss' column is used for the y-axis.
# This line graph shows the change in validation loss across epochs.
plt.plot(monet_losses['Epoch'], monet_losses['Validation Loss'], label='Validation Loss')

# Label the x-axis as 'Epoch' to indicate the training progress.
plt.xlabel('Epoch')
# Label the y-axis as 'Loss' to indicate the value being measured.
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Initialize a new plot with a width of 10 inches and a height of 6 inches
plt.figure(figsize=(10, 6))

# Plot the training loss for each epoch.
# 'Epoch' column from 'real_losses' DataFrame is the x-axis, showing the training progress over time.
# 'Training Loss' column is the y-axis, representing the magnitude of loss during training.
plt.plot(real_losses['Epoch'], real_losses['Training Loss'], label='Training Loss')
# Plot the validation loss for each epoch alongside the training loss.
# 'Validation Loss' shows how well the model is performing on unseen data over the epochs.
plt.plot(real_losses['Epoch'], real_losses['Validation Loss'], label='Validation Loss')
# Set the label for the x-axis to 'Epoch' to indicate the time dimension of the training process.
plt.xlabel('Epoch')
# Set the label for the y-axis to 'Loss' indicating the metric being measured and optimized.
plt.ylabel('Loss')
# Add a title to the graph to describe what is being displayed.
plt.title('Training and Validation Losses')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
def generate_images(generator, img):
    """
    Generates an image using the trained generator model.
    
    Parameters:
        generator: The trained generator model.
        img: The input image for generating a new image.
    
    Returns:
        The generated image from the model.
    """
    # Add an extra dimension to the image making it suitable for the model (batch size of 1)
    img = np.expand_dims(img, axis=0)

    # Use the generator model to predict the output image from the input image
    generated_img = generator.predict(img)[0]  # [0] to get the first item from the batch

    return generated_img

# Iterate through one batch from the test dataset
for real_batch, monet_batch in test_dataset.take(1):
    # Process and display the first real image from the batch
    real_image = encode_image(real_batch[0])  # Decode image to displayable format
    plt.imshow(real_image)
    plt.title("Real")
    plt.axis("off")  # Hide the axis
    plt.show(block=False)  # Display the image without blocking execution

    # Generate an image from the real image using the trained real-to-Monet generator
    real_generate = generate_images(real_gen_trained, real_batch[0])
    real_generate_img = encode_image(real_generate)  # Decode the generated image for display
    plt.imshow(real_generate_img)
    plt.title("real_generate")
    plt.axis("off")
    plt.show(block=False)

    print("-" * 60)  # Print a separator line

    # Process and display the first Monet-style image from the batch
    monet_image = encode_image(monet_batch[0])
    plt.imshow(monet_image)
    plt.title("Monet")
    plt.axis("off")
    plt.show(block=False)

    # Generate an image from the Monet image using the trained Monet-to-real generator
    monet_generate = generate_images(monet_gen_trained, monet_batch[0])
    monet_generate_img = encode_image(monet_generate)
    plt.imshow(monet_generate_img)
    plt.title("monet_generate")
    plt.axis("off")
    plt.show(block=False)


# step 2


In [ ]:
class CycleGan(tf.keras.Model):
    def __init__(self, generator_real2monet, generator_monet2real, discriminator_real, discriminator_monet, lr=5e-4, beta_1=0.5):
        # Initialize the parent class
        super(CycleGan, self).__init__()

        # Assign the provided generators and discriminators to the class instance
        self.generator_real2monet = generator_real2monet  # Converts real images to Monet style
        self.generator_monet2real = generator_monet2real  # Converts Monet images to real style
        self.discriminator_real = discriminator_real      # Distinguishes between real and generated real images
        self.discriminator_monet = discriminator_monet    # Distinguishes between Monet and generated Monet images
        self.lr = lr  # Learning rate for the optimizers
        self.beta_1 = beta_1  # Beta_1 parameter for the Adam optimizer

        
    def compile(self, mac=True):
        # Compile method customizes the training process
        # The 'mac' parameter is used to choose between custom or standard Adam optimizer
        optimizer = tf.keras.optimizers.Adam(learning_rate=self.lr, beta_1=self.beta_1) if not mac else legacy.Adam(learning_rate=self.lr, beta_1=self.beta_1)

        # Compile all generators and discriminators with the same optimizer
        self.generator_real2monet.compile(optimizer=optimizer)
        self.generator_monet2real.compile(optimizer=optimizer)
        self.discriminator_real.compile(optimizer=optimizer)
        self.discriminator_monet.compile(optimizer=optimizer)
        
    def adversarial_loss(self, discriminator, generated):
        # Computes the loss for fake images to fool the discriminator
        fake_output = discriminator(generated)  # Discriminator's prediction on generated images
        return tf.reduce_mean(tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output))

    
    def cycle_consistency_loss(self, real_images, reconstructed_images):
        # Computes the cycle consistency loss to enforce forward and backward consistency
        return tf.reduce_mean(tf.abs(real_images - reconstructed_images))
    
    def identity_loss(self, real_image, generated_image):
        # Computes the identity loss to preserve color and composition between input and generated images
        return tf.reduce_mean(tf.square(real_image - generated_image))
    
    def discriminator_loss(self, real_output, fake_output):
        # Binary Cross Entropy (BCE) loss is used to distinguish between real and fake images.
        bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    
        # Labels for real images are ones, and for fake images are zeros.
        real_labels = tf.ones_like(real_output)
        fake_labels = tf.zeros_like(fake_output)
    
        # Calculate the loss for the real and fake images separately.
        real_loss = bce_loss(real_labels, real_output)  # Loss for correctly identifying real images
        fake_loss = bce_loss(fake_labels, fake_output)  # Loss for correctly identifying fake images
    
        # Total discriminator loss is the sum of real and fake losses.
        total_loss = real_loss + fake_loss
    
        return total_loss

    
    def train_step(self, batch_data):
        # Unpack the real images and Monet-styled images from the batch data
        batch_real, batch_monet = batch_data
        
        # Open a GradientTape to record the operations for automatic differentiation
        with tf.GradientTape(persistent=True) as tape:
            # Generate fake Monet images from real images and fake real images from Monet images
            fake_monet = self.generator_real2monet(batch_real, training=True)
            fake_real = self.generator_monet2real(batch_monet, training=True)
    
            # Generate reconstructed real and Monet images for cycle consistency loss
            reconstr_real = self.generator_monet2real(fake_monet, training=True)
            reconstr_monet = self.generator_real2monet(fake_real, training=True)
    
            # Generate images for identity loss calculation
            same_real = self.generator_monet2real(batch_real, training=True)
            same_monet = self.generator_real2monet(batch_monet, training=True)
    
            # Calculate the adversarial loss for both generators
            adv_loss_R2M = self.adversarial_loss(self.discriminator_monet, fake_monet)
            adv_loss_M2R = self.adversarial_loss(self.discriminator_real, fake_real)
    
            # Calculate the cycle consistency loss for both real and Monet cycles
            cycle_loss = self.cycle_consistency_loss(batch_real, reconstr_real) + \
                         self.cycle_consistency_loss(batch_monet, reconstr_monet)
    
            # Calculate the identity loss to preserve the color and composition of the input images
            identity_loss_real = self.identity_loss(batch_real, same_real)
            identity_loss_monet = self.identity_loss(batch_monet, same_monet)
    
            # Total generator loss includes adversarial, cycle consistency, and identity losses
            total_gen_R2M_loss = 2 * adv_loss_R2M + cycle_loss + 2 * identity_loss_monet
            total_gen_M2R_loss = 2 * adv_loss_M2R + cycle_loss + 2 * identity_loss_real
    
            # Evaluate the real and fake images through the discriminators
            disc_real_real_output = self.discriminator_real(batch_real, training=True)
            disc_real_fake_output = self.discriminator_real(fake_real, training=True)
            disc_monet_real_output = self.discriminator_monet(batch_monet, training=True)
            disc_monet_fake_output = self.discriminator_monet(fake_monet, training=True)
    
            # Calculate the discriminator loss for real and Monet discriminators
            disc_real_loss = self.discriminator_loss(disc_real_real_output, disc_real_fake_output)
            disc_monet_loss = self.discriminator_loss(disc_monet_real_output, disc_monet_fake_output)
        
        # Compute gradients for generators and discriminators
        gen_R2M_gradients = tape.gradient(total_gen_R2M_loss, self.generator_real2monet.trainable_variables)
        gen_M2R_gradients = tape.gradient(total_gen_M2R_loss, self.generator_monet2real.trainable_variables)
        disc_real_gradients = tape.gradient(disc_real_loss, self.discriminator_real.trainable_variables)
        disc_monet_gradients = tape.gradient(disc_monet_loss, self.discriminator_monet.trainable_variables)
    
        # Apply the gradients to the respective models
        self.generator_real2monet.optimizer.apply_gradients(zip(gen_R2M_gradients, self.generator_real2monet.trainable_variables))
        self.generator_monet2real.optimizer.apply_gradients(zip(gen_M2R_gradients, self.generator_monet2real.trainable_variables))
        self.discriminator_real.optimizer.apply_gradients(zip(disc_real_gradients, self.discriminator_real.trainable_variables))
        self.discriminator_monet.optimizer.apply_gradients(zip(disc_monet_gradients, self.discriminator_monet.trainable_variables))
    
        # Return a dictionary of the computed losses
        return {
            "gen_R2M_loss": total_gen_R2M_loss,
            "gen_M2R_loss": total_gen_M2R_loss,
            "disc_real_loss": disc_real_loss,
            "disc_monet_loss": disc_monet_loss
        }
    
    def test_step(self, batch_data):
        # Unpack real and Monet images from the batch data provided to the method.
        batch_real, batch_monet = batch_data
        
        # Generate fake Monet images from real images and vice versa, setting training to False to avoid updating batch statistics.
        fake_monet = self.generator_real2monet(batch_real, training=False)
        fake_real = self.generator_monet2real(batch_monet, training=False)
    
        # Reconstruct the original domain images from the fakes to compute the cycle consistency loss.
        reconstr_real = self.generator_monet2real(fake_monet, training=False)
        reconstr_monet = self.generator_real2monet(fake_real, training=False)
          
        # Generate images from the real and Monet domains to calculate the identity loss.
        same_real = self.generator_monet2real(batch_real, training=False)
        same_monet = self.generator_real2monet(batch_monet, training=False)
    
        # Calculate adversarial losses to evaluate how well the generators are fooling the discriminators.
        adv_loss_R2M = self.adversarial_loss(self.discriminator_monet, fake_monet)
        adv_loss_M2R = self.adversarial_loss(self.discriminator_real, fake_real)
    
        # Compute cycle consistency loss to ensure that the input image can be reconstructed after a round-trip transformation.
        cycle_loss = self.cycle_consistency_loss(batch_real, reconstr_real) + \
                     self.cycle_consistency_loss(batch_monet, reconstr_monet)
    
        # Compute identity loss to ensure the generator preserves the original image when no translation is needed.
        identity_loss_real = self.identity_loss(batch_real, same_real)
        identity_loss_monet = self.identity_loss(batch_monet, same_monet)
    
        # Calculate the total generator losses, combining the adversarial, cycle, and identity losses.
        total_gen_R2M_loss = 2*adv_loss_R2M + cycle_loss + 2*identity_loss_monet
        total_gen_M2R_loss = 2*adv_loss_M2R + cycle_loss + 2*identity_loss_real
    
        # Evaluate the discriminators' performances on distinguishing real images from fake ones.
        disc_real_real_output = self.discriminator_real(batch_real, training=False)
        disc_real_fake_output = self.discriminator_real(fake_real, training=False)
        disc_monet_real_output = self.discriminator_monet(batch_monet, training=False)
        disc_monet_fake_output = self.discriminator_monet(fake_monet, training=False)
    
        # Compute the total discriminator losses for real and Monet images.
        disc_real_loss = self.discriminator_loss(disc_real_real_output, disc_real_fake_output)
        disc_monet_loss = self.discriminator_loss(disc_monet_real_output, disc_monet_fake_output)
    
        # Return a dictionary of the computed losses for monitoring and evaluation.
        return {
            "gen_R2M_loss": total_gen_R2M_loss,
            "gen_M2R_loss": total_gen_M2R_loss,
            "disc_real_loss": disc_real_loss,
            "disc_monet_loss": disc_monet_loss
        }

# check for GPU

In [ ]:
print("GPU Available:", tf.config.list_physical_devices('GPU'))

In [ ]:
# Initialize two instances of the Generator model for real-to-Monet and Monet-to-real conversions.
generator_real2monet, generator_monet2real = Generator(), Generator()

# Uncomment below lines to load pretrained weights if available
# generator_real2monet.load_weights('pre_train/generator_r2m.h5')
# generator_monet2real.load_weights('pre_train/generator_m2r.h5')

# Initialize two instances of the Discriminator model for real and Monet images.
discriminator_real, discriminator_monet = Discriminator(), Discriminator()

# Set the learning rate and beta_1 parameter for the Adam optimizer.
lr = 0.0001
beta_1 = 0.5

# Create directories to save the models' weights and results.
models_folder = f"models/lr_{lr}_beta_{beta_1}"
os.makedirs(models_folder, exist_ok=True)  # Create the models directory if it doesn't exist
os.makedirs("results", exist_ok=True)  # Create the results directory if it doesn't exist

# Initialize the CycleGan model with the previously created generator and discriminator instances.
cycle_gan_model = CycleGan(generator_real2monet, generator_monet2real, discriminator_real, discriminator_monet, lr=lr, beta_1=beta_1)

# Compile the CycleGAN model to set up the optimizers.
cycle_gan_model.compile()

# Define the number of epochs to train the model and the interval for evaluation.
num_epochs = 30
eval_interval = 2  

# Initialize a dictionary to log the training and evaluation metrics.
log_data = {
    'Epoch': [],
    'Gen R2M Loss Train': [], 'Gen M2R Loss Train': [], 
    'Disc Real Loss Train': [], 'Disc Monet Loss Train': [],
    'Gen R2M Loss Eval': [], 'Gen M2R Loss Eval': [], 
    'Disc Real Loss Eval': [], 'Disc Monet Loss Eval': []
}

# Start the training process.
for epoch in range(1, num_epochs + 1):
    # Dictionary to collect losses for each batch in the current epoch.
    train_losses = {
        'Gen R2M Loss': [], 'Gen M2R Loss': [], 
        'Disc Real Loss': [], 'Disc Monet Loss': []
    }

    # Loop through the training dataset in batches.
    for batch_real, batch_monet in tqdm(train_dataset, desc=f'Epoch {epoch}'):
        # Perform a single training step and return the losses.
        train_logs = cycle_gan_model.train_step((batch_real, batch_monet))

        # Aggregate the losses for later calculation of average loss.
        train_losses['Gen R2M Loss'].append(train_logs['gen_R2M_loss'])
        train_losses['Gen M2R Loss'].append(train_logs['gen_M2R_loss'])
        train_losses['Disc Real Loss'].append(train_logs['disc_real_loss'])
        train_losses['Disc Monet Loss'].append(train_logs['disc_monet_loss'])

    # Calculate the average loss for each metric over all training batches in the current epoch.
    avg_train_losses = {k + ' Train': np.mean(v) for k, v in train_losses.items()}
    
    # Save the models' weights after each epoch.
    cycle_gan_model.generator_real2monet.save_weights(os.path.join(models_folder, f'generator_real2monet_epoch_{epoch}.h5'))
    cycle_gan_model.generator_monet2real.save_weights(os.path.join(models_folder, f'generator_monet2real_epoch_{epoch}.h5'))
    cycle_gan_model.discriminator_real.save_weights(os.path.join(models_folder, f'discriminator_real_epoch_{epoch}.h5'))
    cycle_gan_model.discriminator_monet.save_weights(os.path.join(models_folder, f'discriminator_monet_epoch_{epoch}.h5'))
    
    # Perform evaluation at specified intervals.
    if epoch % eval_interval == 0:
        # Dictionary to collect evaluation metrics.
        eval_metrics = {
            'Gen R2M Loss': 0.0, 'Gen M2R Loss': 0.0, 
            'Disc Real Loss': 0.0, 'Disc Monet Loss': 0.0
        }
        num_batches = 0

        # Loop through the validation dataset in batches.
        for batch_real_eval, batch_monet_eval in tqdm(val_dataset, desc=f'Eval at Epoch {epoch}'):
            # Perform an evaluation step and return the losses.
            eval_logs = cycle_gan_model.test_step((batch_real_eval, batch_monet_eval))

            # Accumulate the losses for later calculation of average loss.
            eval_metrics['Gen R2M Loss'] += eval_logs['gen_R2M_loss']
            eval_metrics['Gen M2R Loss'] += eval_logs['gen_M2R_loss']
            eval_metrics['Disc Real Loss'] += eval_logs['disc_real_loss']
            eval_metrics['Disc Monet Loss'] += eval_logs['disc_monet_loss']
            num_batches += 1

        # Calculate the average loss for each metric over all evaluation batches.
        avg_eval_metrics = {k + ' Eval': v / num_batches for k, v in eval_metrics.items()}

        # Append the current epoch's average training and evaluation metrics to the log data.
        log_data['Epoch'].append(epoch)
        for key, value in {**avg_train_losses, **avg_eval_metrics}.items():
            log_data[key].append(value)

        print(log_data)  # Optionally print the log data to monitor the training and evaluation progress.

# Convert the collected log data into a pandas DataFrame for easier handling and visualization.
log_df = pd.DataFrame(log_data)

# Save the compiled log data to a CSV file for further analysis and review.
log_df.to_csv(f'results/log_lr_{lr}_beta_{beta_1}.csv', index=False)

In [ ]:
def plot_losses(df_losses):
    epochs = df_losses['Epoch']
    gen_R2M_loss_train = df_losses['Gen R2M Loss Train']
    gen_M2R_loss_train = df_losses['Gen M2R Loss Train']
    disc_real_loss_train = df_losses['Disc Real Loss Train']
    disc_monet_loss_train = df_losses['Disc Monet Loss Train']
    gen_R2M_loss_eval = df_losses['Gen R2M Loss Eval']
    gen_M2R_loss_eval = df_losses['Gen M2R Loss Eval']
    disc_real_loss_eval = df_losses['Disc Real Loss Eval']
    disc_monet_loss_eval = df_losses['Disc Monet Loss Eval']

    # Plot the data
    plt.figure(figsize=(12, 8))

    # Plot training loss
    plt.plot(epochs, gen_R2M_loss_train, label='Gen R2M Loss Train', marker='o')
    plt.plot(epochs, gen_M2R_loss_train, label='Gen M2R Loss Train', marker='o')
    plt.plot(epochs, disc_real_loss_train, label='Disc Real Loss Train', marker='o')
    plt.plot(epochs, disc_monet_loss_train, label='Disc Monet Loss Train', marker='o')

    # Plot evaluation loss
    plt.plot(epochs, gen_R2M_loss_eval, label='Gen R2M Loss Eval', marker='o')
    plt.plot(epochs, gen_M2R_loss_eval, label='Gen M2R Loss Eval', marker='o')
    plt.plot(epochs, disc_real_loss_eval, label='Disc Real Loss Eval', marker='o')
    plt.plot(epochs, disc_monet_loss_eval, label='Disc Monet Loss Eval', marker='o')

    # Add labels and title
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Over Epochs')
    plt.legend()
    plt.grid(True)

    # Show plot
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_images(generator, img):
    # Add a batch dimension to the image, making it suitable for the generator model
    img = np.expand_dims(img, axis=0)

    # Generate an image using the provided generator model, then remove the batch dimension
    generated_img = generator.predict(img)[0]

    return generated_img

def plot_predictions(test_dataset, generator_monet2real, generator_real2monet):
    # Take one batch from the test dataset
    for real_batch, monet_batch in test_dataset.take(1):
        # Decode and display the first real image from the batch
        real_image = encode_image(real_batch[0])
        plt.imshow(real_image)
        plt.title("Real")
        plt.axis("off")
        plt.show(block=False)  # Display the image without blocking the execution

        # Generate a Monet-style image from the real image and display it
        monet_generate = generate_images(generator_real2monet, real_batch[0])
        monet_generate_img = encode_image(monet_generate)
        plt.imshow(monet_generate_img)
        plt.title("Monet Generated")
        plt.axis("off")
        plt.show(block=False)

        # Convert the Monet-generated image back to real style and display it
        real_generate = generate_images(generator_monet2real, monet_generate)
        real_generate_img = encode_image(real_generate)
        plt.imshow(real_generate_img)
        plt.title("Real Regenerated")
        plt.axis("off")
        plt.show(block=False)

        # Print a separator line
        print("-" * 60)

        # Decode and display the first Monet image from the batch
        monet_image = encode_image(monet_batch[0])
        plt.imshow(monet_image)
        plt.title("Monet")
        plt.axis("off")
        plt.show(block=False)

        # Generate a real-style image from the Monet image and display it
        real_generate = generate_images(generator_monet2real, monet_batch[0])
        real_generate_img = encode_image(real_generate)
        plt.imshow(real_generate_img)
        plt.title("Real Generated")
        plt.axis("off")
        plt.show(block=False)

        # Convert the real-generated image back to Monet style and display it
        monet_generate = generate_images(generator_real2monet, real_generate)
        monet_generate_img = encode_image(monet_generate)
        plt.imshow(monet_generate_img)
        plt.title("Monet Regenerated")
        plt.axis("off")
        plt.show(block=False)


In [ ]:
df_losses_identity = pd.read_csv('results/log_lr_0.0001_beta_0.5_identity.csv')
print(df_losses_identity.head())

df_losses_pretrain_identity = pd.read_csv('results/log_lr_0.0001_beta_0.5_pretrain_identity.csv')
print(df_losses_pretrain_identity.head())

df_losses_pretrain = pd.read_csv('results/log_lr_0.0001_beta_0.5_pretrain.csv')
print(df_losses_pretrain.head())

# loss plot

In [ ]:
plot_losses(df_losses_identity)

In [ ]:
plot_losses(df_losses_pretrain_identity)

In [ ]:
plot_losses(df_losses_pretrain)

In [ ]:
# Define the epoch number of the best-performing model to use for loading weights.
best_epoch = 30

# Specify the folder where the model weights are saved.
models_folder = "models/lr_0.0001_beta_0.5_identity"

# Create instances of the Generator model for both real-to-Monet and Monet-to-real image translation.
generator_real2monet, generator_monet2real = Generator(), Generator()

# Load the weights of the best-performing model for the real-to-Monet generator from the specified epoch.
generator_real2monet.load_weights(os.path.join(models_folder, f'generator_real2monet_epoch_{best_epoch}.h5'))

# Similarly, load the weights of the best-performing model for the Monet-to-real generator from the same epoch.
generator_monet2real.load_weights(os.path.join(models_folder, f'generator_monet2real_epoch_{best_epoch}.h5'))

# Print a statement indicating that predictions are being made using the models from the specified batch number.
print("identity batch 30")

# Visualize the predictions using the loaded generator models on a set of images from the test dataset.
plot_predictions(test_dataset, generator_monet2real, generator_real2monet)


In [ ]:
# Define the epoch number from which to load the trained model weights. This indicates the chosen "best" model.
best_epoch = 20

# Specify the directory where the models trained with specific hyperparameters are saved.
models_folder = "models/lr_0.0001_beta_0.5_pretrain"

# Initialize two Generator instances: one for converting real images to Monet style, and another for the reverse process.
generator_real2monet, generator_monet2real = Generator(), Generator()

# Load the weights into the real-to-Monet generator model from the specified epoch, indicating this model's parameters produced satisfactory results at this stage of training.
generator_real2monet.load_weights(os.path.join(models_folder, f'generator_real2monet_epoch_{best_epoch}.h5'))

# Similarly, load the weights into the Monet-to-real generator model from the same epoch, ensuring consistency in performance evaluation.
generator_monet2real.load_weights(os.path.join(models_folder, f'generator_monet2real_epoch_{best_epoch}.h5'))

# Output to indicate that the predictions being made use the models from the specified epoch, here associated with a "pretraining" phase or strategy.
print("pretrain batch 20")

# Call the function to visualize predictions. This function demonstrates the generators' abilities by converting test dataset images between real and Monet styles and displaying these transformations.
plot_predictions(test_dataset, generator_monet2real, generator_real2monet)


In [ ]:
# Set the epoch number to identify the best model performance for loading the weights.
best_epoch = 20

# Specify the directory where the trained models' weights are stored, indicating it's for a pre-trained model with identity loss.
models_folder = "models/lr_0.0001_beta_0.5_pretrain_identity"

# Initialize two Generator models, one for converting real images to Monet style and the other for Monet to real.
generator_real2monet, generator_monet2real = Generator(), Generator()

# Load the weights for the real-to-Monet generator from the specified best epoch within the given models folder.
generator_real2monet.load_weights(os.path.join(models_folder, f'generator_real2monet_epoch_{best_epoch}.h5'))

# Load the weights for the Monet-to-real generator from the same epoch, indicating these models have been pre-trained with identity loss consideration.
generator_monet2real.load_weights(os.path.join(models_folder, f'generator_monet2real_epoch_{best_epoch}.h5'))

# Print a statement to signify that predictions are being made using the pre-trained models with identity loss at the 20th batch.
print("pretrain_identity batch 20")

# Display the predictions by converting images from the test dataset between real and Monet styles using the loaded generator models.
plot_predictions(test_dataset, generator_monet2real, generator_real2monet)

## evel best model

In [ ]:
# need to change all this 

In [ ]:
best_epoch = 30
models_folder = "models/lr_0.0001_beta_0.5_identity"
generator_real2monet, generator_monet2real = Generator(),Generator()
generator_real2monet.load_weights(os.path.join(models_folder, f'generator_real2monet_epoch_{best_epoch}.h5'))
generator_monet2real.load_weights(os.path.join(models_folder, f'generator_monet2real_epoch_{best_epoch}.h5'))


In [ ]:
import numpy as np
import tensorflow as tf
from scipy.spatial.distance import cosine

# Function to calculate FID
def calculate_fid(mu_real, sigma_real, mu_gen, sigma_gen):
    fid = np.sum((mu_real - mu_gen)**2) + np.trace(sigma_real + sigma_gen - 2 * np.sqrt(np.dot(sigma_real, sigma_gen)))
    return fid

# Function to calculate memorization distance
def calculate_memorization_distance(real_samples, generated_samples):
    distances = []
    for gen_sample in generated_samples:
        min_distance = np.inf
        for real_sample in real_samples:
            distance = cosine(gen_sample, real_sample)
            if distance < min_distance:
                min_distance = distance
        distances.append(min_distance)
    return np.mean(distances)

In [ ]:

# Load a pre-trained Inception model for feature extraction
inception_model = tf.keras.applications.InceptionV3(include_top=False, pooling='avg')

real_images_features_all = []
monet_images_features_all = []

generated_monet_features_all = []
generated_real_features_all = []

# Loop over the dataset
for real_batch, monet_batch in test_dataset:
    real_images_features = inception_model.predict(encode_image(real_batch))
    monet_images_features = inception_model.predict(encode_image(monet_batch))

    generated_monet_features = inception_model.predict(generator_real2monet(encode_image(real_batch) , training = False))
    generated_real_features = inception_model.predict(generator_monet2real(encode_image(monet_batch) , training = False))

    # Accumulate features for the entire dataset
    real_images_features_all.append(real_images_features)
    monet_images_features_all.append(monet_images_features)
    generated_monet_features_all.append(generated_monet_features)
    generated_real_features_all.append(generated_real_features)

# Concatenate features from all batches
real_images_features_all = np.concatenate(real_images_features_all)
monet_images_features_all = np.concatenate(monet_images_features_all)
generated_monet_features_all = np.concatenate(generated_monet_features_all)
generated_real_features_all = np.concatenate(generated_real_features_all)





In [ ]:
# Calculate mean and covariance of features for real images
mu_real = np.mean(real_images_features_all, axis=0)
sigma_real = np.cov(real_images_features_all, rowvar=False)

# Calculate mean and covariance of features for monet images
mu_monet = np.mean(monet_images_features_all, axis=0)
sigma_monet = np.cov(monet_images_features_all, rowvar=False)

# Calculate mean and covariance of features for generated Monet images
mu_gen_monet = np.mean(generated_monet_features_all, axis=0)
sigma_gen_monet = np.cov(generated_monet_features_all, rowvar=False)

# Calculate mean and covariance of features for generated real images
mu_gen_real = np.mean(generated_real_features_all, axis=0)
sigma_gen_real = np.cov(generated_real_features_all, rowvar=False)

# Compute FID scores
fid_score_monet = calculate_fid(mu_monet, sigma_monet, mu_gen_monet, sigma_gen_monet)
fid_score_real = calculate_fid(mu_real, sigma_real, mu_gen_real, sigma_gen_real)

# Compute memorization distance for generated Monet images
memorization_distance_monet = calculate_memorization_distance(real_images_features_all, generated_monet_features_all)

# Compute memorization distance for generated real images
memorization_distance_real = calculate_memorization_distance(real_images_features_all, generated_real_features_all)

# Threshold memorization distances based on epsilon
epsilon = 0.1
if memorization_distance_monet > epsilon:
    memorization_distance_monet = 1.0

if memorization_distance_real > epsilon:
    memorization_distance_real = 1.0

# Calculate MiFID scores
mifid_score_monet = fid_score_monet * memorization_distance_monet
mifid_score_real = fid_score_real * memorization_distance_real

print("FID Score (Monet):", fid_score_monet)
print("Memorization Distance (Monet):", memorization_distance_monet)
print("MiFID Score (Monet):", mifid_score_monet)

print("FID Score (Real):", fid_score_real)
print("Memorization Distance (Real):", memorization_distance_real)
print("MiFID Score (Real):", mifid_score_real)


In [ ]:
print("sefe")

# save results for kaggle 


In [ ]:
!mkdir ../images

In [ ]:
directory_path = "data/photo_jpg/"
image_list = load_images_from_dir(directory_path)
print("Number of images loaded:", len(image_list))

In [ ]:
best_epoch = 30
models_folder = "models/lr_0.0001_beta_0.5_identity"
generator_real2monet = Generator()
generator_real2monet.load_weights(os.path.join(models_folder, f'generator_real2monet_epoch_{best_epoch}.h5'))


In [ ]:
i = 1
for image in image_list:
    real_image = decode_image(image)
    
    monet_generate = generate_images(generator_real2monet , real_image)
    monet_generate_img = encode_image(monet_generate).numpy()
    monet_generate_img_fixed_size = cv2.resize(monet_generate_img, (256, 256))
    # print(monet_generate_img_fixed_size.shape)
    # plt.imshow(monet_generate_img_fixed_size)
    # plt.title("Real")
    # plt.axis("off")
    # plt.show(block = False)
  
    cv2.imwrite(f"../images/{i}.jpg", monet_generate_img_fixed_size)
    i += 1
    

In [ ]:
images_folder_path = '../images'  
zip_filename = 'images.zip'
shutil.make_archive(zip_filename.split('.')[0], 'zip', images_folder_path)
print(f"Images folder has been successfully zipped to {zip_filename}.")
